In [2]:
import random
import time
import math
import numpy as np
import matplotlib.pyplot as plt

import copy # to copy lists for python2.7

try:
    from .individual_single_mode import Individual
except Exception: #ImportError
    from individual_single_mode import Individual

In [3]:
############################### EVALFLAG.PY ##########################################
## A method to evaluate the quality of flagging.

## Inputs : data, flags
## Output : some metric of quality.

## Test : Run tfcrop/rflag with diff params, saving flagversions
##           Extract the 2D flags (perhaps just from flagversions)

VIS_DIR = '/Users/bjuncklaus/Dropbox/Evolutionary/Data/datasets/'
VIS_FILENAME = 'FewScans_G55_Spw2_Scan50.ms'
# VIS_FILENAME = 'FewScans_G55_Spw6_Scan6_HanningSmooth.ms'
# VIS_FILENAME = 'Four_ants_3C286.ms_Spw9_Scan30.ms'
# VIS_FILENAME = 'G29_Spw0_Scan145.ms'
# VIS_FILENAME = 'G29_Spw0_Scan38.ms'
# VIS_FILENAME = 'G29_Spw7_Scan83.ms'
# TODO - put the % Flagged on the best fit
# VIS_FILENAME = 'FewScans_G55_Spw6_Scan52_HanningSmooth.ms'
# VIS = '/home/vega2/bmartins/datasets/FewScans_G55_Spw2_Scan4.ms'
# VIS = '/Users/bjuncklaus/Dropbox/Evolutionary/Data/datasets/FewScans_G55_Spw6_Scan4_HanningSmooth.ms'
VIS = VIS_DIR + VIS_FILENAME

In [4]:
def runtest(cmdlist=[]):
    ## G55 dataset
    ## scans 4,6 are 3C286
    ## scans 50,52 are G55 SNR
    ## spws 6 and 11 have good representative RFI.


    # vis = '/home/vega2/bmartins/datasets/FewScans_G55_Spw7_Scan4.ms'

    # vis = '../Data/Four_ants_3C286.ms'

    spw = '0'
    scan = '4'
    vname = 'v1'

    ## Run the flagging
    flagdata(vis=VIS, mode='unflag', flagbackup=False)

    if (not cmdlist):
    #     tfcrop_params = "spw='" + spw + "' scan='" + scan + "'" + cmdlist[0]
    #     extend_params = "spw='" + spw + "' scan='" + scan + "'" + cmdlist[1]
    #     cmdlist = [tfcrop_params, extend_params]
    # else:
        # cmdlist = ["spw='"+spw+"' scan='"+scan+"' mode='tfcrop' maxnpieces=4 freqcutoff=3.0 usewindowstats='sum' " ,
        #              "spw='"+spw+"' scan='"+scan+"' mode='extend' growaround=True growtime=60.0" ]
        cmdlist = ["' mode='tfcrop'"]  # default value

    print()
    print("CMDLIST:", cmdlist)

    flagdata(vis=VIS, mode='list', inpfile=cmdlist, flagbackup=False)
    # flagmanager(vis=vis, mode='save', versionname=vname)

    ## Read the flags
    # flagmanager(vis=vis, mode='restore', versionname=vname)
    dat = getvals(col='DATA', vis=VIS)
    flag = getvals(col='FLAG', vis=VIS)

    # plotit(dat, flag)

    flag_percentage = np.sum(flag) / (1.0 * np.prod(flag.shape)) * 100.0
    print('% Flagged : ', flag_percentage)
    print('VIS : ', VIS_FILENAME)

    score = calcquality(dat, flag)
    if (math.isnan(score)):
        return float("inf"), flag_percentage

    return score, flag_percentage

In [5]:
def calcquality(dat, flag):
    """ Need to minimize the score that it returns"""

    shp = dat.shape

    npts = 0
    sumsq = 0.0
    maxval = 0.0
    leftover = []
    flagged = []
    for chan in range(0, shp[1]):
        for tm in range(0, shp[2]):
            val = np.abs(dat[0, chan, tm])
            if flag[0, chan, tm] == False:
                leftover.append(val)
            else:
                flagged.append(val)

    dmax, dmean, dstd = printstats(np.abs(dat[0, :, :]))
    rmax, rmean, rstd = printstats(leftover)
    fmax, fmean, fstd = printstats(flagged)

    maxdev = (rmax - rmean) / rstd
    fdiff = fmean - rmean
    sdiff = fstd - rstd

    print("Max deviation after flagging : ", maxdev)
    print("Diff in mean of flagged and unflagged : ", fdiff)
    print("Std after flagging : ", rstd)

    aa = np.abs(np.abs(maxdev) - 3.0)
    bb = 1.0 / ((np.abs(fdiff) - rstd) / rstd)
    cc = 1.0 / (np.abs(sdiff) / rstd)
    dd = 0.0

    pflag = (len(flagged) / (1.0 * shp[1] * shp[2])) * 100.0
    #
    # if pflag > 95.0:  # Check if what's flagged really looks like RFI.
    #     ## Mean and std should look similar...
    #     dd = (fmean - fstd) / fstd

    if pflag > 75.0:  # Check if what's flagged really looks like RFI.
        ## More flags means a worse score...
        dd = (pflag - 75.0) / 10.0

    res = np.sqrt(aa ** 2 + bb ** 2 + cc * 2 + dd * 2)

    if (fdiff < 0.0):
        res = res + res + 10.0

    print("Score : ", res)

    return res

In [6]:
def printstats(arr):
    if (len(arr) == 0):
        return 0, 0, 1

    med = np.median(arr)
    std = np.std(arr)
    maxa = np.max(arr)
    mean = np.mean(arr)
    # print 'median : ', med
    # print 'std : ', std
    # print 'max : ', maxa
    # print 'mean : ', mean
    # print " (Max - mean)/std : ", ( maxa - mean ) / std

    return maxa, mean, std

In [7]:
def getvals(col='DATA', vis="", spw="", scan=""):

    # print("SPW:", spw, "DDID:", ddid)

    tb.open(vis)
    if (spw and scan):
        tb.open(vis + '/DATA_DESCRIPTION')
        spwids = tb.getcol('SPECTRAL_WINDOW_ID')
        ddid = str(np.where(spwids == eval(spw))[0][0])
        tb1 = tb.query('SCAN_NUMBER==' + scan + ' && DATA_DESC_ID==' + ddid + ' && ANTENNA1=1 && ANTENNA2=2')
    else:
        tb1 = tb.query('ANTENNA1=1 && ANTENNA2=2')
    dat = tb1.getcol(col)
    tb1.close()
    tb.close()
    return dat

In [1]:
def plotit(dat, flag):
    pl.clf()
    pl.subplot(121)
    pl.imshow(np.abs(dat[0, :, :]))
    pl.subplot(122)
    pl.imshow(np.abs(dat[0, :, :] * (1 - flag[0, :, :])))